---
# Base de Datos MySQL con Python

---

---
## 1. Test de funcionamiento

En primer lugar vamos a intentar conectarnos con alguna de las base de datos con las que estuve trabajando
con anterioridad con el MySQL Workbench.

Vamos a ver la version que estamos utilizando

---

In [5]:
!pip install pymysql
import pymysql


def mysqlconnect():
    # To connect MySQL database
    conn = pymysql.connect(
        host='localhost',
        user='root',
        password="16deEnero",
        db='Henry_05',
    )

    cur = conn.cursor()
    cur.execute("select @@version")
    output = cur.fetchall()
    print(output)

    # To close the connection
    conn.close()


mysqlconnect()


(('8.0.31',),)


---
## 2. Vamos a armar una funcion que nos permita crear una base de datos

La contraseña, nombre de usuario y direccion del host estan almacenadas en un archivo aparte llamado db_config.

Dentro de db_config hay algo similar a esto:

```python
config = {
    "password": "unPassword",
    "host" :"localhost",
    "user": "root"
}
```

---

In [6]:
import pymysql
import db_config as c


# Funcion para crear una base de datos
def crearBaseDatos(c, db_name):
    # To connect MySQL database
    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"]
        
    )

    cur = conn.cursor()

    # Select query
    cur.execute(f"create database {db_name}")
    
    conn.close()


# Creo una base de datos llamada oferta gastronomica
#crearBaseDatos(c, "oferta_gastronomica")


---
## 3. Vamos a armar una funcion que nos permita crear una tabla

---

In [7]:
def crearTabla(c, db_name, table_name, column_info):
    '''
        column_info ="""id int not null auto_increment,
            nombre varchar(40), 
            categoria varchar(60), 
            direccion varchar(100), 
            barrio varchar(40), 
            comuna varchar(40), 
            primary key(id)"""
        
        db_name = "nombre_de_la_bd"

        table_name = "nombre_de_la_tabla"
    '''

    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"],
        db=db_name
    )

    cur = conn.cursor()

    # Select query
    cur.execute(f"create table {table_name}({column_info}) ")

    conn.close()

# col_info ="""id int not null auto_increment,
# nombre varchar(40), 
# categoria varchar(60), 
# direccion varchar(40), 
# barrio varchar(40), 
# comuna varchar(40), 
# primary key(id)"""

# crearTabla(c, "oferta_gastronomica", "oferta_gastronomica2", col_info)

---
### Funcion para agregar un unico registro

---

In [8]:
def agregarRegistro(c, db_name, table_name, columnas, datos):
    """
    Agrega un unico registro a la base de datos
    """
    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"],
        db=db_name
    )

    cur = conn.cursor()

    # select query
    query = f"insert into {table_name} ({columnas}) VALUES (%s,%s,%s,%s,%s);"
    cur.execute(query, datos )
    conn.commit()
    conn.close()

# columnas = "nombre, categoria, direccion, barrio, comuna"
# datos = ["bar de pepe", "bares", "calle falsa 123", "Flores", "Chiapas"]
# agregarRegistro(c, "oferta_gastronomica", "oferta_gastronomica", columnas, datos)

---
### Funcion para agregar varios registros en simultaneo

---

In [9]:
def agregarRegistros(c, db_name, table_name, columnas, datos):
    """
    Agrega varios registros a la base de datos
    """
    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"],
        db=db_name
    )

    cur = conn.cursor()

    # select query
    query = f"insert into {table_name} ({columnas}) VALUES (%s,%s,%s,%s,%s);"
    cur.executemany(query, datos )
    conn.commit()
    conn.close()

---
## Creamos una funcion para mostrar los registros de una tabla

---

In [22]:
def showRegistros(c, db_name, table_name, columnas):
    """
    Muestra los registros
    """
    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"],
        db=db_name
    )

    cur = conn.cursor()

    # select query
    query = f"select {columnas} from {table_name}"
    cur.execute(query)   
    myresult = cur.fetchall()
    
    conn.close()

    return myresult

---
## Query SELECT con WHERE clause


---

In [26]:
def queryGenerica(c, db_name, table_name, queryG):
    """
    Muestra registros con condicion
    """
    conn = pymysql.connect(
        host= c.config["host"],
        user=c.config["user"],
        password=c.config["password"],
        db=db_name
    )

    cur = conn.cursor()

    # select query
    query = queryG
    cur.execute(query)   
    myresult = cur.fetchall()
    
    conn.close()

    return myresult

---
## Resolucion del ejercicio

---

### Primero accedemos al repositorio y guardamos la info en una variable

---

In [3]:
import requests

import pymysql
from io import StringIO
import csv

bajada = requests.get('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')
bajada.encoding = 'utf-8'
bajada = bajada.text
documento = csv.reader(StringIO(bajada))
next(documento)
locales = [(linea[3], linea[4], linea[13], linea[14], linea[15][7:]) for linea in documento]

In [4]:
locales

[('GUANTANAMERA',
  'RESTAURANTE',
  'MOREAU DE JUSTO, ALICIA AV. 2092',
  'Puerto Madero',
  '1'),
 ('1843 VINACOTECA', 'VINERIA', 'HUMBERTO 1º 1843', 'San Cristobal', '3'),
 ('2080 EXPRESS', 'RESTAURANTE', 'JURAMENTO AV. 2080', 'Belgrano', '13'),
 ('22', 'CAFE', 'MURGUIONDO 4188', 'Villa Lugano', '8'),
 ('4 EN 2 I', 'RESTAURANTE', 'BORGES, JORGE LUIS 2095', 'Palermo', '14'),
 ('4 X 4 CAFE', 'BAR', 'DEL LIBERTADOR AV. 3838', 'Palermo', '14'),
 ('5TA AVENIDA', 'RESTAURANTE', 'SARMIENTO 814', 'San Nicolas', '1'),
 ('5TA AVENIDA I', 'RESTAURANTE', 'SARMIENTO 381', 'San Nicolas', '1'),
 ('5TA ZONA', '', 'MURILLO 1049', 'Villa Crespo', '15'),
 ('650', 'RESTAURANTE', 'SAN MARTIN 650', 'San Nicolas', '1'),
 ('788 FOODBAR', 'RESTAURANTE', 'ARENALES 1877', 'Recoleta', '2'),
 ('7 SEVEN', 'RESTAURANTE', 'GORRITI 5802', 'Palermo', '14'),
 ('8 MILIMETROS', 'RESTAURANTE', 'GORRITI 5535', 'Palermo', '14'),
 ('9 DE JULIO', 'RESTAURANTE', 'PELLEGRINI, CARLOS 593', 'San Nicolas', '1'),
 ('9 DE JULIO II

In [15]:
locales[640:650]

[("CUTTER'S CAFE",
  'RESTAURANTE',
  'PUEYRREDON, HONORIO, DR. AV. y JUSTO, JUAN B. AV.',
  'Villa Crespo',
  '15'),
 ("D7'A SEPTIMO DIA",
  'BAR',
  'CARRANZA, ANGEL JUSTINIANO 1875',
  'Palermo',
  '14'),
 ('DACROY', 'RESTAURANTE', 'ARENALES 2399', 'Recoleta', '2'),
 ('DADA', 'RESTAURANTE', 'SAN MARTIN 941', 'Retiro', '1'),
 ("DADY'S", 'RESTAURANTE', 'PASO 415', 'Balvanera', '3'),
 ('DAFNI', 'RESTAURANTE', 'ARMENIA 1231', 'Palermo', '14'),
 ('DALI', 'RESTAURANTE', 'JUNIN 1950', 'Recoleta', '2'),
 ('DALINGER', 'RESTAURANTE', 'GORRITI 5801', 'Palermo', '14'),
 ('DAMBLEE', 'RESTAURANTE', 'RIVADAVIA AV. 3402', 'Almagro', '5'),
 ('DAMIANO', 'BAR', 'GORRITI 5577', 'Palermo', '14')]

---
## Creamos una base de datos llamada oferta_gastronomica

---

In [16]:

db_name = "oferta_gastronomica"
crearBaseDatos(c, db_name)


---
## Creamos una tabla llamada (tambien) oferta_gastronomica

---

In [17]:

column_info ="""id int not null auto_increment,
            nombre varchar(40), 
            categoria varchar(60), 
            direccion varchar(100), 
            barrio varchar(40), 
            comuna varchar(40), 
            primary key(id)"""

db_name = "oferta_gastronomica"

table_name = "oferta_gastronomica"

crearTabla(c, db_name, table_name, column_info)         

---
## Agregamos los registros guardados en la lista locales

---

In [18]:
db_name = "oferta_gastronomica"
table_name = "oferta_gastronomica"
columnas = "nombre, categoria, direccion, barrio, comuna"
datos = locales
agregarRegistros(c, db_name, table_name, columnas, datos)

---
## Mostramos los registros agregados

---

In [24]:
db_name = "oferta_gastronomica"
table_name = "oferta_gastronomica"
columnas = "nombre, categoria, direccion, barrio, comuna"

query = showRegistros(c, db_name, table_name, columnas)


for registros in query:
    print(registros)

('GUANTANAMERA', 'RESTAURANTE', 'MOREAU DE JUSTO, ALICIA AV. 2092', 'Puerto Madero', '1')
('1843 VINACOTECA', 'VINERIA', 'HUMBERTO 1º 1843', 'San Cristobal', '3')
('2080 EXPRESS', 'RESTAURANTE', 'JURAMENTO AV. 2080', 'Belgrano', '13')
('22', 'CAFE', 'MURGUIONDO 4188', 'Villa Lugano', '8')
('4 EN 2 I', 'RESTAURANTE', 'BORGES, JORGE LUIS 2095', 'Palermo', '14')
('4 X 4 CAFE', 'BAR', 'DEL LIBERTADOR AV. 3838', 'Palermo', '14')
('5TA AVENIDA', 'RESTAURANTE', 'SARMIENTO 814', 'San Nicolas', '1')
('5TA AVENIDA I', 'RESTAURANTE', 'SARMIENTO 381', 'San Nicolas', '1')
('5TA ZONA', '', 'MURILLO 1049', 'Villa Crespo', '15')
('650', 'RESTAURANTE', 'SAN MARTIN 650', 'San Nicolas', '1')
('788 FOODBAR', 'RESTAURANTE', 'ARENALES 1877', 'Recoleta', '2')
('7 SEVEN', 'RESTAURANTE', 'GORRITI 5802', 'Palermo', '14')
('8 MILIMETROS', 'RESTAURANTE', 'GORRITI 5535', 'Palermo', '14')
('9 DE JULIO', 'RESTAURANTE', 'PELLEGRINI, CARLOS 593', 'San Nicolas', '1')
('9 DE JULIO II', 'CAFE', 'RIESTRA AV. 5498', 'Villa

---
## Muestra barrios con mas pubs

---


In [27]:
db_name = "oferta_gastronomica"
table_name = "oferta_gastronomica"
queryG = """
SELECT categoria, barrio, count(*) as cantidad
from oferta_gastronomica
GROUP BY barrio, categoria
HAVING categoria = 'PUB'
ORDER BY cantidad desc;
"""
registros = queryGenerica(c, db_name, table_name, queryG)

for registro in registros:
    print(registro)

('PUB', 'Recoleta', 8)
('PUB', 'Caballito', 6)
('PUB', 'Villa Devoto', 5)
('PUB', 'Flores', 4)
('PUB', 'Retiro', 4)
('PUB', 'Mataderos', 4)
('PUB', 'San Nicolas', 3)
('PUB', 'Palermo', 3)
('PUB', 'Villa Del Parque', 2)
('PUB', 'Balvanera', 2)
('PUB', 'Villa Ortuzar', 2)
('PUB', 'Agronomia', 2)
('PUB', 'Liniers', 2)
('PUB', 'NuÃƒÂ±ez', 2)
('PUB', 'Villa Santa Rita', 1)
('PUB', 'Monserrat', 1)
('PUB', 'Villa Pueyrredon', 1)
('PUB', 'Villa Lugano', 1)



---

## Cantidad de locales por rubro

---

In [28]:
db_name = "oferta_gastronomica"
table_name = "oferta_gastronomica"

queryG = """
#Locales por rubro
SELECT categoria, count(*) as cantidad
from oferta_gastronomica
GROUP BY categoria
ORDER BY cantidad desc;
"""

registros = queryGenerica(c, db_name, table_name, queryG)

for registro in registros:
    print(registro)

('RESTAURANTE', 1990)
('CAFE', 334)
('BAR', 247)
('', 116)
('CONFITERIA', 55)
('PUB', 53)
('VINERIA', 22)
('SANDWICHERIA', 4)
('DELIVERY & TAKE AWAY', 2)


---
## Cantidad de restorantes por comuna

---

In [29]:


db_name = "oferta_gastronomica"
table_name = "oferta_gastronomica"

queryG = """
SELECT comuna, count(*) as cantidad
from oferta_gastronomica
GROUP BY comuna, categoria
HAVING categoria = 'RESTAURANTE'
ORDER BY cantidad desc;
"""

registros = queryGenerica(c, db_name, table_name, queryG)

for registro in registros:
    print(registro)

('1', 609)
('14', 366)
('2', 227)
('3', 145)
('13', 108)
('4', 95)
('6', 87)
('7', 64)
('15', 57)
('5', 56)
('9', 52)
('10', 43)
('12', 42)
('11', 33)
('8', 6)
